In [1]:
!pip3 install https://github.com/EleutherAI/lm-evaluation-harness

     - 495.6 kB 12.3 MB/s 0:00:00
  ERROR: Cannot unpack file /tmp/pip-unpack-pcjqax86/lm-evaluation-harness (downloaded from /tmp/pip-req-build-q2eovs25, content-type: text/html; charset=utf-8); cannot detect archive format
ERROR: Cannot determine archive format of /tmp/pip-req-build-q2eovs25

[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip3 install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 14.0 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
from huggingface_hub import notebook_login

/home/debian/source/repos/BIMPoC/pythonenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
!pip3 install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 2.2 MB/s eta 0:00:0031m1.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 21.8 MB/s eta 0:00:00m eta 0:00:010:01

[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [6]:
notebook_login()

In [9]:
!lm_eval --tasks list

2024-07-10:18:21:32,130 INFO     [__main__.py:272] Verbosity set to INFO
2024-07-10:18:21:32,530 INFO     [__init__.py:403] `group` and `group_alias` keys in tasks' configs will no longer be used in the next release of lm-eval. `tag` will be used to allow to call a collection of tasks just like `group`. `group` will be removed in order to not cause confusion with the new ConfigurableGroup which will be the offical way to create groups with addition of group-wide configuations.
2024-07-10:18:21:37,016 INFO     [__main__.py:321] Available Tasks:
 - 20_newsgroups
 - aclue
 - aclue_ancient_chinese_culture
 - aclue_ancient_literature
 - aclue_ancient_medical
 - aclue_ancient_phonetics
 - aclue_basic_ancient_chinese
 - aclue_couplet_prediction
 - aclue_homographic_character_resolution
 - aclue_named_entity_recognition
 - aclue_poetry_appreciate
 - aclue_poetry_context_prediction
 - aclue_poetry_quality_assessment
 - aclue_poetry_sentiment_analysis
 - aclue_polysemy_resolution
 - aclue_readin

In [ ]:
!lm_eval --model hf \
    --model_args pretrained=microsoft/Phi-3-mini-4k-instruct \
    --tasks truthfulqa \
    --device cpu \
    --batch_size 6 \
    --output_path ./results \
    --log_samples


2024-07-10:22:16:51,869 INFO     [__main__.py:272] Verbosity set to INFO
2024-07-10:22:16:52,294 INFO     [__init__.py:403] `group` and `group_alias` keys in tasks' configs will no longer be used in the next release of lm-eval. `tag` will be used to allow to call a collection of tasks just like `group`. `group` will be removed in order to not cause confusion with the new ConfigurableGroup which will be the offical way to create groups with addition of group-wide configuations.
2024-07-10:22:16:57,234 INFO     [__main__.py:369] Selected Tasks: ['truthfulqa']
2024-07-10:22:16:57,235 INFO     [evaluator.py:158] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234
2024-07-10:22:16:57,235 INFO     [evaluator.py:195] Initializing hf model, with arguments: {'pretrained': 'microsoft/Phi-3-mini-4k-instruct'}
/home/debian/source/repos/BIMPoC/pythonenv/lib/python3.10/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: CUDA unknown error

In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [68]:
from langchain_google_vertexai import VertexAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.chains import QAGenerationChain
import random
from tqdm import tqdm
from langchain.output_parsers import ResponseSchema, StructuredOutputParser



In [9]:
import wandb

In [100]:

    # Initialize wandb
    wandb.init(project="test")


In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=100)

In [8]:
loader = PyPDFLoader("/home/debian/Downloads/test.pdf")
qa_chunks = loader.load_and_split(text_splitter=text_splitter)
print("Number of chunks for building qa eval set:", len(qa_chunks))


Number of chunks for building qa eval set: 252


In [70]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser

class Joke(BaseModel):
    question: str = Field(description="question")
    answer: str = Field(description="answer to the question")

parser = JsonOutputParser(pydantic_object=Joke)


In [73]:
response_schemas = [
    ResponseSchema(name="question",description="generated question"),
    ResponseSchema(
        name="answer",
        description="answer to the generat.",
    ),
]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)



In [75]:
templ = """You are a smart assistant designed to come up with meaninful question and answer pair. The question should be to the point and the answer should be as detailed as possible.
Given a piece of text, you must come up with a question and answer pair that can be used to evaluate a QA bot. Do not make up stuff. Stick to the text to come up with the question and answer pair.

Please come up with a question/answer pair, in the specified JSON format, for the following text:
{format_instructions}
----------------
{text}"""

PROMPT = PromptTemplate(
    template=templ,
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

In [76]:
random_chunks = []
for i in range(100):
    random_chunks.append(random.randint(5, 172)) 


In [77]:
model = VertexAI(model_name="gemini-pro")
command_chain = QAGenerationChain.from_llm(llm=model, text_splitter=text_splitter)
chain = PROMPT | model | output_parser
qa = chain.invoke(qa_chunks[0].page_content)

In [96]:
qa_pairs = []
for idx in tqdm(random_chunks, desc="QA Generation ..."):
    qa = chain.invoke(qa_chunks[idx].page_content)
    qa["model"] = "gemini-pro"
    qa_pairs.append(qa)


QA Generation ...: 100%|█████████████████████████████████| 100/100 [03:29<00:00,  2.09s/it]


In [82]:
qa = chain.invoke(qa_chunks[0].page_content)

In [98]:
import pandas as pd
qa_df = pd.DataFrame(qa_pairs)
wandb.log({"QA Eval Pair": qa_df})


In [101]:
run = wandb.init(project="test")

artifact = run.use_artifact('tlzl/test/run-7r2egld2-QAEvalPair:v0', type='run_table')
artifact_dir = artifact.download()


wandb:   1 of 1 files downloaded.  


In [104]:
import json
with open(f"{artifact_dir}/QA Eval Pair.table.json") as f:
    data = json.load(f)
columns = data["columns"]
data = data["data"]


In [105]:
data

[['How can CCA be connected to the Alternating Conditional Expectation (ACE) method?',
  "Nonlinear CCA's solution can be connected to the Alternating Conditional Expectation (ACE) method. This was discovered by Makur et al. [2015]. Painsky et al. [2020] further refine this connection, leading to theoretical improvements for CCA.",
  'gemini-pro'],
 ['Explain one of the popular frameworks for learning from unlabeled data?',
  'One popular framework uses a weakly trained network to apply pseudolabels to images, and then train using these labels in a standard supervised fashion.',
  'gemini-pro'],
 ['What is the purpose of the linear and MLP probes used in the figure?',
  'The linear and MLP probes are used to predict the ImageNet-1k labels from the output of a Resnet50 backbone during SimCLR training (online) and post-training (offline).',
  'gemini-pro'],
 ['What is the main difference between DINO and iBOT?',
  "DINO's objective is to reconstruct the image pixels, while iBOT's objecti

In [106]:
eval_df = pd.DataFrame(columns=columns, data=data)
eval_df = eval_df.sample(frac=1).reset_index(drop=True)


In [108]:
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings

vector_store = Chroma.from_documents(documents=qa_chunks, embedding=GoogleGenerativeAIEmbeddings(model="models/embedding-001"),persist_directory="db")

In [109]:
retriever = vector_store.as_retriever()

In [115]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

prompt = PromptTemplate.from_template(
            """
            <s> [INST] Answer the users question based on the retrieved context. If you do not find an answer in the context, respond with i dont know[/INST] </s> 
            [INST] Question: {question} 
            Context: {context} 
            Answer: [/INST]
            """
        )


fchain = ({"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser())


In [116]:
print(fchain.invoke("How can CCA be connected to the Alternating Conditional Expectation (ACE) method?"))


The CCA framework seeks two transformations U = f_x(X) and V = f_y(Y) such that  
 $L=-\frac{1}{N} \sum_{n=1}^N \langle U_n, V_n \rangle$  
 where U_n and V_n are zero-mean representations,  
 $\frac{1}{NU_T} U U = \frac{1}{NV_T} V V = I$ 
 with d (the dimension of the output mappings) such that d <= min(dim(X), dim(Y)). 

 

 Nonlinear CCA was achieved by Breiman and Friedman [1985] in the univariate output setting, and by Makur et al. [2015] in the multivariate output setting, by connecting the solution to eq. (13) to the Alternating Conditional Expectation (ACE) method. 

 

 The optimal representation for nonlinear CCA using the Alternating Conditional Expectation proves new theoretical bounds that lead to further refinements of CCA. These ideas were extended to deep learning in Deep Canonically Correlated Autoencoders (DCCAE) an autoencoder regularized via CCA.


In [118]:
    predictions = []

    for idx, tmp_df in tqdm(eval_df.iterrows(), desc="Generating Answers"):
        question = tmp_df.question
        response = fchain.invoke(question)

        predictions.append({"response": response})


Generating Answers: 100it [06:00,  3.61s/it]


In [119]:
predictions

[{'response': "## Learning Localized Features Without Annotations in Semi-Supervised Learning for Dense Prediction Tasks\n\nWhile semi-supervised learning (SSL) algorithms excel in image classification tasks, their effectiveness in dense prediction tasks, which require pixel-level localization, has been debated. This is mainly due to the lack of segmentation or detection annotations in SSL, making it challenging to train models that can accurately identify and locate objects within images.\n\nHowever, researchers have proposed various strategies to overcome this hurdle and learn localized features without annotations. Here are some notable approaches:\n\n**1. Modifications to SSL routines:**\n\n* **Masked Autoencoding:** This technique involves masking parts of the image and training the model to reconstruct the masked regions. This forces the model to focus on specific areas of the image, promoting the development of localized features.\n* **Jigsaw Puzzles:** In this method, the image

In [121]:
from langchain.evaluation.qa import QAEvalChain

qa_pairs = eval_df.to_dict("records")
eval_chain = QAEvalChain.from_llm(llm = model)
graded_outputs = eval_chain.evaluate(
    qa_pairs, predictions, question_key="question", prediction_key="response"
)


In [132]:
graded_outputs

[{'results': "## Grade: **CORRECT** \n\n## Explanation: \n\nThe student answer provides a comprehensive overview of various methods employed to learn localized features without annotations during semi-supervised learning (SSL) for dense prediction tasks. It accurately identifies the limitations of SSL algorithms for such tasks and highlights several strategies developed to overcome these limitations. Additionally, the student's response emphasizes the importance of weakly-supervised data and leveraging transformers for improving localization accuracy. \n\nWhile the True Answer correctly states that specific methods modifying SSL routines are employed for localization, it does not provide details of these methods. The student's answer, however, delves deeper into specific strategies and approaches, demonstrating a thorough understanding of the topic. \n\nTherefore, considering the comprehensive and accurate explanation provided by the student, the answer is graded as **CORRECT**. \n"},


In [135]:
    correct = 0
    for graded_output in graded_outputs:
        assert isinstance(graded_output, dict)
        if "Grade: CORRECT" in graded_output["results"]: 
            correct+=1


In [136]:
correct

18

In [137]:
    llm_accuracy = (correct/len(graded_outputs))*100
    wandb.log({"llm_based_eval_acc": llm_accuracy}, commit=False)


In [138]:
llm_accuracy

18.0

In [140]:
from evaluate import load
squad_metric = load("squad")


In [141]:
    for i, eg in enumerate(qa_pairs):
        eg["id"] = str(i)
        eg["answers"] = {"text": [eg["answer"]], "answer_start": [0]}
        predictions[i]["id"] = str(i)
        predictions[i]["prediction_text"] = predictions[i]["response"]



In [142]:
    for p in predictions:
        del p["response"]


In [143]:
    new_qa_pairs = qa_pairs.copy()
    for eg in new_qa_pairs:
        del eg["question"]
        del eg["answer"]
        del eg["model"]


In [147]:
    import numpy as np
    exact_matches = []
    f1s = []

    for qa_pair, prediction in zip(new_qa_pairs, predictions):
        result = squad_metric.compute(
            references=[qa_pair],
            predictions=[prediction],
        )
        exact_matches.append(result["exact_match"])
        f1s.append(result["f1"])

    mean_exact_match = np.mean(exact_matches)
    mean_f1 = np.mean(f1s)


In [148]:
mean_f1

25.70535311865371

In [149]:
    eval_df["Prediction"] = predictions
    eval_df["LLM Evaluating LLM"] = graded_outputs
    eval_df["Exact Match"] = exact_matches
    eval_df["F1"] = f1s

    wandb.log({"QA Eval Result": eval_df})


In [150]:
    incorrect = 0
    for graded_output in graded_outputs:
        assert isinstance(graded_output, dict)
        if "CORRECT" in graded_output["results"]: 
            incorrect+=1


In [151]:
incorrect

98